In [ ]:
pip install jmespath parsel unidecode pyodbc

In [1]:
import json
import requests
import jmespath
from parsel import Selector
import os

In [21]:
import requests

url = "https://www.nettruyenup.vn/tim-truyen"

headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
}

response = requests.get(url, headers=headers)
print(response.text)

<!DOCTYPE html> <html id="ctl00_Html1" lang="vi"> <head id="ctl00_Head1" itemscope="" itemtype="http://schema.org/WebPage"> <meta http-equiv="Content-Type" content="text/html; charset=utf-8" /> <title> NetTruyen - Tìm truyện tranh online </title> <meta name="keywords" content="NetTruyen, Truyện tranh, Truyen tranh online, Đọc truyện tranh, Truyện tranh hot, Truyện tranh hay, Truyện ngôn tình, truyenqq, mi2manga, doctruyen3q, cmanga, blogtruyen, truyentranhaudio, hamtruyen, truyensieuhay, nettruyenmax, anettruyen"> <meta name="description" content="❶✔️ Tìm truyện tranh - Tất cả truyện đều có thể tìm thấy tại NetTruyen" /> <meta property="og:image" content="https://st.ntcdntempv3.com/data/sites/1/share.jpg" /> <meta property="og:title" content="NetTruyen - Tìm truyện tranh online" /> <meta property="og:site_name" content="NetTruyen" /> <meta property="og:url" content="https://www.nettruyenup.vn" /> <meta property="og:type" content="website" /> <meta property="og:description" content="❶✔️

In [2]:
def SaveJSON(fileName,JsonComic):
  print(fileName+"\n")
  with open(f"{fileName}.json", "w") as outfile:
      json.dump(JsonComic, outfile,indent= 2)

In [3]:
f = open('mapComic.json')
ListComic = json.load(f)

In [31]:
for idx,oneComic in  enumerate(ListComic["comic"]):
    flag = os.path.isfile(f"ComicData/{oneComic['urlComic']}.json")
    if(not flag) :
        # continue
        print(idx,oneComic['urlComic'])
    # f = open(f"ComicData/{oneComic['urlComic']}.json")
    # data = json.load(f)
    # if (not data['URL']):
    #     print(idx,oneComic['urlComic'])
    #     f.close()
    #     os.remove (f"ComicData/{oneComic['urlComic']}.json")

732 ke-khung-bo
2282 a-story-about-a-very-ordinary-couple
6154 cham-soc-cho-chi-dau
6337 cham-soc-cho-chi-dau
8711 xe-ban-thuc-an-dien-loan
9038 purple-hyacinth
10513 kimi-wa-maru-de-ano-hana-no-you-de
11762 jungle-wa-itsumo-hale-nochi-guu
12071 mahoutsukai-to-deshi-no-futekisetsu-na-kankei
12074 giao-chu-tan-nhiem-muon-hoan-luong
12075 toi-ganh-team-mot-minh
12076 toi-co-thuoc-hoi-han
12077 majo-senpai-nichijou
12127 shinigamihime-no-saikon
12135 haru-no-ogawa-wa-dokidoki-suru-yo
12136 truyen-sims-write-up-your-life-with-color
12284 hong-trang-chuoc-chuoc
12285 tatakae-ry-zanpaku-shij-saiky-no-deshi
12289 the-gladiator
12293 chom-sao-bac-cuc-cua-chung-ta
12314 onose-to-sugoshita
12316 con-rong-hau-gai
12493 miunohri-to-swan-co-nang-xinh-dep
12727 chim-yen-khong-ve
12772 nhom-nhac-thien-ha-lich
14979 nhom-nhac-thien-ha-lich
15512 dendrobates
15692 choegang-geunyeo
16122 te-bao-than-quoc
16854 jungle-wa-itsumo-hale-nochi-guu
17717 chi-ton-tho-hao-he-thong


In [38]:
allcomicstring ="""
        {
          "comic": props.pageProps.dataComicNew[*].
          {
            urlComic:urlComic,
            totalChapter:totalChapter,
            createDateComic:createDateComic,
            lastUpdateChapter:lastUpdateChapter
          }
        }
"""
def GetAllComicMapJSON():
  ListComic = {"comic":[]}
  expression = jmespath.compile( allcomicstring )
  for idx in range(1, 600):
    response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/tim-truyen?status=2&sort=2&page={idx}')
    if(response.status_code != 200):
      print(f'Fetching Error Page{idx} {response.status_code}')
      continue
    print(f'Fetching Page{idx} {response.status_code}')
    selector = Selector(text=response.text)
    json_data = selector.css("script[type='application/json']::text").get()
    data = json.loads(json_data)
    flat_data = expression.search(data)
    if(flat_data["comic"] == []): break
    ListComic["comic"].extend(flat_data["comic"])
  return ListComic


In [ ]:
ListComic = GetAllComicMapJSON()

In [50]:
page_not_found = []
comic_not_found = []

In [53]:
def GetComicJSON(urlComic):
  global comic_not_found
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}')
  if (response.status_code != 200):
    print(response.status_code)
    comic_not_found.append(urlComic)
    return None
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()

  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Title": props.pageProps.comic[0].nameComic,
        "URL": props.pageProps.comic[0].urlComic,
        "CoverImage":   props.pageProps.comic[0].imgComic,
        "CreateAt":  props.pageProps.comic[0].create_date
        "statusComic": props.pageProps.comic[0].statusComic
        "Category":   props.pageProps.comic[0].category[*].name_cate,
        "LastChapter": props.pageProps.comic[0].lastChapter
        "TotalChapter": props.pageProps.comic[0].totalChapter
        "Chapters": props.pageProps.listChapter[0].{urlChapter:urlChapter, dateUpdateChapter: dateUpdateChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data


def fetchData(start = 0, end = 100):
    global ListComic
    for idx,oneComic in  enumerate(ListComic["comic"][start:end]):
        urlComic = oneComic["urlComic"]
        JsonComic = GetComicJSON(urlComic)
        if (JsonComic == None): continue
        print(f"({idx},{JsonComic['Title']})")
        SaveJSON(f"ComicData/{urlComic}",JsonComic)

In [60]:
for i in range(100,1100,100):
    print(i)

100
200
300
400
500
600
700
800
900
1000


In [ ]:
import threading
for i in range(100,10000,100):
    th1 = threading.Thread(target=fetchData, args=(i, i+100))
    th1.start()

In [31]:
SaveJSON(f"mapComic",ListComic)

mapComic



In [9]:
def GetChapterOfComicJSON(urlComic, chapter=1):
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}/chapter-{chapter}')
  print(response.status_code)
  
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()
  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Page": props.pageProps.dataContentChapter[*].{indexImg:indexImg,imgChapter:imgChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data

In [18]:
import unidecode
import re
def CreateSlug(accented_string:str):

  # accented_string is of type 'unicode'
  unaccented_string = unidecode.unidecode(accented_string).lower()
  unaccented_string = re.sub(r"[^\w\s]", '', unaccented_string)
  unaccented_string=re.sub(r"\s+","-",unaccented_string)
  return unaccented_string

CreateSlug("Tu Ti\u00ean \u1ede Th\u1ebf Gi\u1edbi Si\u00eau N\u0103ng L\u1ef1c")

'tu-tien-o-the-gioi-sieu-nang-luc'

In [ ]:
from bs4 import BeautifulSoup

# Assuming we have an HTML document in the 'html_content' variable
soup = BeautifulSoup(html_content, 'html.parser')

# Find the desired element and extract its text using get_text()
elements = soup.find_all('span',itemprop="name")

#get_text with strip set to true
for idx,element in  enumerate(elements):
  print(f"({idx},{element.text})")

In [ ]:
import pyodbc 
server = 'DESKTOP-33SUFGJ\\SQLEXPRESS'
database = 'ComicApp'
username = 'your_username'
password = 'your_password'
connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};'

In [75]:
connectionString = 'C:/Users/ACER/Desktop/ComicApp/SQL/ComicDB.db'
conn = sqlite3.connect(connectionString)
cursor = conn.cursor()

In [23]:
import glob

listPath = glob.glob("ComicData/*")
len(listPath)

20633

# Query all Genre

In [76]:
SQL_QUERY_GENRE = """
  Select * from GENRE
"""
cursor.execute(SQL_QUERY_GENRE)
allGenre = cursor.fetchall()
dictGenre = {}
for genre in allGenre:
  slug = CreateSlug(genre[1])
  dictGenre[slug] = genre[0]
dictGenre

OperationalError: database is locked

# Insert genre for comic

In [21]:
def FindComicID(cursor,urlComic):
  #sql query
  SQL_QUERY = f"""
  Select * from COMIC where URL = '{urlComic}'
  """
  cursor.execute(SQL_QUERY)
  comic = cursor.fetchone()
  if(comic):
    return comic[0]
  return None

In [91]:
SQL_QUERY = """
 INSERT INTO COMIC_GENRE (genreid, comicid) VALUES
  (?,?)
"""
for path in listPath:
  f = open(path)
  datajson = json.load(f)
  if(not datajson['URL']):
    print(path)
    continue
  for genre in datajson['Category']:
    slug = CreateSlug(genre)
    if(slug not in dictGenre): 
      continue
    genreid = dictGenre[slug]
    comicid = FindComicID(cursor,datajson['URL'])
    if(comicid == None): continue
    VALUES =(genreid,comicid)
    cursor.execute(SQL_QUERY,VALUES)
    conn.commit()

ComicData\kakunaru-ue-ha.json
ComicData\ma-nhan-hoa-son.json
ComicData\rakuraku-shinwa.json
ComicData\yuusha-sama-ni-ikinari-kyuukonsareta-no-desu-ga.json


In [77]:
conn.commit()

In [90]:
SQL_QUERY = """
  INSERT INTO COMIC (Title,URL,CoverImage,Status,CreateAt,Rating) VALUES
  (?,?,?,?,?,?)
"""
for path in listPath:
  f = open(path)
  datajson = json.load(f)
  if(not datajson['URL']):
    print(path)
    continue
  VALUES =(datajson['Title'],datajson['URL'],datajson['CoverImage'],datajson['statusComic'],datajson['CreateAt'],10)
  cursor.execute(SQL_QUERY,VALUES)
  conn.commit()
# cursor.close()


IntegrityError: UNIQUE constraint failed: COMIC.URL

In [78]:
SQL_QUERY = """
  INSERT INTO COMIC (Title,URL,CoverImage,Status,CreateAt,Rating) VALUES
  {0}
"""
print(SQL_QUERY.format(values))


  INSERT INTO COMIC (Title,URL,CoverImage,Status,CreateAt,Rating) VALUES
  ('0.0 Mhz - Tần Số Chết', '0-0-mhz-tan-so-chet', '0-0-mhz-tan-so-chet.jpg', 0, '10/01/2024', 10)



In [88]:
cursor.execute(SQL_QUERY.format(values))
conn.commit()

IntegrityError: UNIQUE constraint failed: COMIC.URL

In [11]:
SQL_QUERY_GENRE = """
  Select * from GENRE
"""
cursor.execute(SQL_QUERY_GENRE)
allGenre = cursor.fetchall()

In [12]:
allGenre

[(1,
  'Actio',
  'Thể loại này thường có nội dung về đánh nhau, bạo lực, hỗn loạn, với diễn biến nhanh'),
 (2, 'Adult', 'Thể loại có đề cập đến vấn đề nhạy cảm, chỉ dành cho tuổi 17+'),
 (3,
  'Adventure',
  'Thể loại phiêu lưu, mạo hiểm, thường là hành trình của các nhân vật'),
 (4, 'Anime', 'Truyện đã được chuyển thể thành film Anime'),
 (5,
  'Chuyển Sinh',
  'Thể loại này là những câu chuyện về người ở một thế giới này xuyên đến một thế giới khác, có thể là thế giới mang phong cách trung cổ với kiếm sĩ và ma thuật, hay thế giới trong game, hoặc có thể là bạn chết ở nơi này và được chuyển sinh đến nơi khác'),
 (6,
  'Comedy',
  'Thể loại có nội dung trong sáng và cảm động, thường có các tình tiết gây cười, các xung đột nhẹ nhàng'),
 (7, 'Comic', 'Truyện tranh Châu Âu và Châu Mĩ'),
 (8, 'Cooking', 'Thể loại có nội dung về nấu ăn, ẩm thực'),
 (9, 'Cổ Đại', 'Truyện có nội dung xảy ra ở thời cổ đại phong kiến.'),
 (10,
  'Doujinshi',
  'Thể loại truyện phóng tác do fan hay có thể cả nh

In [6]:
sqliteConnection.close()

In [ ]:
"a-story-about-a-very-ordinary-couple.json"
"cham-soc-cho-chi-dau.json"
"chi-ton-tho-hao-he-thong.json"
"chim-yen-khong-ve.json"
"dendrobates.json"
"giao-chu-tan-nhiem-muon-hoan-luong.json"
"haru-no-ogawa-wa-dokidoki-suru-yo.json"
"hong-trang-chuoc-chuoc.json"
"jungle-wa-itsumo-hale-nochi-guu.json"
"kakunaru-ue-ha.json"
"ke-khung-bo.json"
"kimi-wa-maru-de-ano-hana-no-you-de.json"
"ma-nhan-hoa-son.json"
"mahoutsukai-to-deshi-no-futekisetsu-na-kankei.json"
"miunohri-to-swan-co-nang-xinh-dep.json"
"nhom-nhac-thien-ha-lich.json"
"onose-to-sugoshita.json"
"purple-hyacinth.json"
"rakuraku-shinwa.json"
"te-bao-than-quoc.json"
"the-gladiator.json"
"toi-co-thuoc-hoi-han.json"
"toi-ganh-team-mot-minh.json"
"truyen-sims-write-up-your-life-with-color.json"
"xe-ban-thuc-an-dien-loan.json"
"yuusha-sama-ni-ikinari-kyuukonsareta-no-desu-ga.json"